In [2]:
import pandas as pd
import numpy as np

In [3]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [4]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')

## 13. 연습 문제

### 13.1. 집계 분석 전처리

#### 집계 분석 준비

2016년 숙박의 예약 경향을 연령대x성별로 집계
- 연령은 10세 단위로 구분
- 60세 이상은 하나로 묶음
- 해당 인원수, 합계 예약 횟수, 평균 예약 인원 수, 평균 예약 단가

In [5]:
customer_tb['age_rank'] = customer_tb['age'].apply(
    lambda x: int(x / 10) * 10
)
customer_tb['age_rank'] = pd.Categorical(customer_tb['age_rank'])
customer_tb['age_rank'].cat.add_categories(['60+'], inplace=True)
customer_tb.loc[
    customer_tb['age_rank'].isin([60.0, 70.0, 80.0]),
    'age_rank'
] = '60+'
customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)

In [6]:
reserve_tb = reserve_tb.query('"2016-01-01" <= checkin_date <= "2016-12-31"')

In [7]:
m_tb = pd.merge(
    reserve_tb,
    customer_tb,
    on='customer_id'
)
result = m_tb \
    .groupby(['age_rank', 'sex']) \
    .agg({
        'customer_id': 'nunique',
        'reserve_id': 'nunique',
        'people_num': ['mean', 'sum'],
        'total_price': 'sum',
    })

result.columns = ['_'.join(col).strip() for col in result.columns.values]
result.rename(columns={
    'customer_id_nunique': 'customer_cnt',
    'reserve_id_nunique': 'rsv_cnt',
    'people_num_mean': 'people_num_avg',
}, inplace=True)

result['price_per_person_avg'] = result['total_price_sum'] / result['people_num_sum']
result

customer_cnt  rsv_cnt  people_num_avg  people_num_sum  \
age_rank sex                                                            
20       man               8       22        2.227273              49   
         woman             9       23        2.913043              67   
30       man             111      282        2.609929             736   
         woman           104      266        2.605263             693   
40       man             116      309        2.566343             793   
         woman           116      289        2.512111             726   
50       man              74      189        2.597884             491   
         woman            61      169        2.414201             408   
60+      man             151      382        2.581152             986   
         woman           138      371        2.493261             925   

                total_price_sum  price_per_person_avg  
age_rank sex                                           
20       man            3311300          67577.551020  
         woman          2864500          42753.731343  
30       man           29824900          40522.961957  
         woman         27267400          39346.897547  
40       man           30281000          38185.372005  
         woman         30142300          41518.319559  
50       man           20123000          40983.706721  
         woman         17920700          43923.284314  
60+      man           40616400          41193.103448  
         woman         36779700          39761.837838